# statistic features demo

## matching coefficient

In [1]:
# -*- coding: utf-8 -*-
"""
匹配系数（无向符号网络）
"""


import numpy as np
import networkx as nx
import copy


def divide_degree(G):
    '''首先将原始网络G拆分成正边网络和负边网络
    输入：G原始网络
    输出：p正边数据列表
         n负边数据列表
         Gp正边网络
         Gn负边网络
    '''
    
    edges = G.edges(data = True)
    pos_edges = []  # 正边数据容器
    neg_edges = []  # 负边数据容器

    posdegree_graph = copy.deepcopy(G)  # 拥有正度值网络
    negdegree_graph = copy.deepcopy(G)  # 拥有负度值网络

    for item in edges:
        if item[2]['weight'] == 1:
            pos_edges.append(item)
            negdegree_graph.remove_edge(item[0], item[1])
        if item[2]['weight'] == 2:
            neg_edges.append(item)
            posdegree_graph.remove_edge(item[0], item[1])
                
    return posdegree_graph, negdegree_graph, pos_edges, neg_edges


def sum_jk(Gpdeg, Gndeg, pos_edges, neg_edges):
    '''计算无向符号网络的4种度相关性（公式模块）
         +-- 网络中正连接的两个节点正正度匹配特性
         ++- 网络中正连接的两个节点正负度匹配特性
         -++ 网络中负连接的两个节点正正度匹配特性
         -+- 网络中负连接的两个节点正负度匹配特性
    输入：pos_edges正边数据列表
         neg_edges负边数据列表
         Gpdeg正边网络
         Gndeg负边网络
    输出：jn_kn, jn_kn1, jn_kn2, +--公式模块
         jp_kn, jp_kn1, jp_kn2, ++-公式模块
         jp_kp, jp_kp1, jp_kp2, -++公式模块
         jp_kn3, jp_kn4, jp_kn5，-+-公式模块
    '''
        
    # +--
    jnkn = []
    jnkn1 = []
    jnkn2 = []

    # ++-
    jpkn = []
    jpkn1 = []
    jpkn2 = []

    # -++
    jpkp = []
    jpkp1 = []
    jpkp2 = []

    # -+-
    jpkn3 = [] 
    jpkn4 = []
    jpkn5 = []

    for edge1 in pos_edges:
        
        # +--
        a = int(Gndeg.degree(edge1[0])) * int(Gndeg.degree(edge1[1]))
        jnkn.append(a)
        jn_kn = sum(jnkn)
    
        b = int(Gndeg.degree(edge1[0])) + int(Gndeg.degree(edge1[1]))
        b1 = 0.5*b
        jnkn1.append(b1)
        jn_kn1 = sum(jnkn1)
        
        c = (np.square(int(Gndeg.degree(edge1[0])))
             + np.square(int(Gndeg.degree(edge1[1]))))
        c1 = 0.5*c
        jnkn2.append(c1)
        jn_kn2 = sum(jnkn2)
        
        # ++-        
        o = int(Gpdeg.degree(edge1[0])) * int(Gndeg.degree(edge1[1]))
        jpkn.append(o)
        jp_kn = sum(jpkn)
        
        p = int(Gpdeg.degree(edge1[0])) + int(Gndeg.degree(edge1[1]))
        p1 =  0.5*p
        jpkn1.append(p1)
        jp_kn1 = sum(jpkn1)
        
        q = (np.square(int(Gpdeg.degree(edge1[0]))) 
             + np.square(int(Gndeg.degree(edge1[1]))))
        q1 = 0.5*q
        jpkn2.append(q1)
        jp_kn2 = sum(jpkn2)
        
    for edge2 in neg_edges:
       
        # -++
        d = int(Gpdeg.degree(edge2[0]))*int(Gpdeg.degree(edge2[1]))
        jpkp.append(d)
        jp_kp = sum(jpkp)
        
        e = int(Gpdeg.degree(edge2[0])) + int(Gpdeg.degree(edge2[1]))
        e1 = 0.5*e
        jpkp1.append(e1)
        jp_kp1 = sum(jpkp1)
        
        f = (np.square(int(Gpdeg.degree(edge2[0]))) 
             + np.square(int(Gpdeg.degree(edge2[1]))))
        f1 = 0.5*f
        jpkp2.append(f1)
        jp_kp2 = sum(jpkp2)
        
        # -+-
        r = int(Gpdeg.degree(edge2[0])) * int(Gndeg.degree(edge2[1]))
        jpkn3.append(r)
        jp_kn3=sum(jpkn3)
        
        s = int(Gpdeg.degree(edge2[0])) + int(Gndeg.degree(edge2[1]))
        s1 = 0.5*s
        jpkn4.append(s1)
        jp_kn4 = sum(jpkn4)
        
        t = (np.square(int(Gpdeg.degree(edge2[0]))) 
             + np.square(int(Gndeg.degree(edge2[1]))))
        t1 = 0.5*t
        jpkn5.append(t1)
        jp_kn5 = sum(jpkn5)
    
    return (jn_kn, jn_kn1, jn_kn2, jp_kn, jp_kn1, jp_kn2, 
            jp_kp, jp_kp1, jp_kp2, jp_kn3, jp_kn4, jp_kn5)



def final_formula(a, q, w, e):  
    '''匹配系数计算公式
    输入：a为正边或负边数量
         q,w,e分别为三组公式模块
    输出：formula匹配系数公式（Newman）
    '''
              
    formula = (1/a * q - (1/a*w)*(1/a*w))/(1/a * e - (1/a*w)*(1/a*w))

    return formula


# 代入数值生成网络
G = nx.read_edgelist('N46edge.txt', nodetype=int, data=(('weight', float),))
# 计算各种公式模块数值
Gpdeg, Gndeg, pos_edges, neg_edges = divide_degree(G)
a0, b0, c0, d0, e0, f0, g0, h0, i0, j0, k0, l0 = sum_jk(Gpdeg, Gndeg, 
                                                        pos_edges, neg_edges)
# 组装成各自匹配特性数值
pnn = final_formula(len(pos_edges), a0, b0, c0)
ppn = final_formula(len(pos_edges), d0, e0, f0)
npp = final_formula(len(neg_edges), g0, h0, i0)
npn = final_formula(len(neg_edges), j0, k0, l0)
# 输出各自匹配特性数值
print("The assortativity coefficient of +-- in the original network is:",
      "%3.3f" %pnn)
print("The assortativity coefficient of ++- in the original network is:",
      "%3.3f" %ppn)
print("The assortativity coefficient of -++ in the original network is:",
      "%3.3f" %npp)
print("The assortativity coefficient of -+- in the original network is:",
      "%3.3f" %npn)





The assortativity coefficient of +-- in the original network is: 0.005
The assortativity coefficient of ++- in the original network is: -0.803
The assortativity coefficient of -++ in the original network is: 0.082
The assortativity coefficient of -+- in the original network is: -0.199


## excess average degree

In [2]:
# -*- coding: utf-8 -*-
"""
余平均度
"""


import networkx as nx


def divide_network(G):
    '''将原始网络G拆成正边网络或负边网络（无孤立节点）
    输入：G 原始网络
    输出：Gp 正边网络（无孤立节点）
         Gn 负边网络（无孤立节点）
    '''
    
    edge_list = []
   
    for i, j, weight_data in G.edges(data = True):
        if 'weight' in weight_data:  # 排除孤立节点
            # 权重值去除字典结构  
            edge_list.append([i, j, weight_data['weight']])
            
    positive_edges = []  # 正边数据容器
    negitive_edges = []  # 负边数据容器
    
    for item in edge_list:
        if item[2] == 1:       
            positive_edges.append(item)
        if item[2] == 2:       
            negitive_edges.append(item)
    
    # 组成正边网络（无孤立节点）    
    Gp = nx.Graph()
    Gp.add_weighted_edges_from(positive_edges)  
    
    # 组成负边网络（无孤立节点）
    Gn = nx.Graph()
    Gn.add_weighted_edges_from(negitive_edges)
    
    return Gp, Gn


def knn_to_list(knn_dict):
    '''按升序分离度值与其对应的余平均度为列表
    输入：knn_dict {度值：其对应的余平均度}字典
    输出：knn_degrees 排序后的度值列表
         knn_values 排序后的余平均度列表
    '''
    
    list_keys = list(knn_dict)  # 字典键列表
    knn_list = []
    
    for item in list_keys:
        knn_list.append((int(item), knn_dict[item]))  #字典变元组
    
    # 以度值为比较对象，对其所在的元组排序     
    knn_list_new = sorted(knn_list, key=lambda knn_list : knn_list[0])    
    knn_degrees = []
    knn_values = []
    
    for item in knn_list_new:
        knn_degrees.append(item[0])  # 分离度值
        knn_values.append(item[1])  # 分离余平均度
    
    return knn_degrees, knn_values


# 生成原始网络G
G = nx.read_edgelist('N46edge.txt', nodetype = int, data = (('weight',float),))
# 生成正边网络或负边网络（无孤立节点）
Gp, Gn = divide_network(G)
# 以字典的形式返回正边网络（无孤立节点）中所有度为K的节点的余平均度
knn_p0 = nx.k_nearest_neighbors(Gp)
# 以字典的形式返回负边网络（无孤立节点）中所有度为K的节点的余平均度
knn_n0 = nx.k_nearest_neighbors(Gn)
# 分离正边网络（无孤立节点）的度值与余平均度
knn_p0_degree, knn_p0_values = knn_to_list(knn_p0)
# 分离负边网络（无孤立节点）的度值与余平均度
knn_n0_degree, knn_n0_values = knn_to_list(knn_n0)

print(knn_p0_degree,'\n')
print(knn_p0_values,'\n')
print(knn_n0_degree,'\n')
print(knn_n0_values,'\n')



[1, 3, 4, 5, 6, 8, 11, 12, 14, 19, 20, 31] 

[9.333333333333334, 5.0, 6.25, 6.0, 5.833333333333333, 4.9375, 13.181818181818182, 12.083333333333334, 11.357142857142858, 20.473684210526315, 20.35, 16.806451612903224] 

[1, 2, 3, 4, 5, 6, 9] 

[4.133333333333334, 4.916666666666667, 4.666666666666667, 5.25, 4.7, 3.6666666666666665, 3.7222222222222223] 



## clustering coefficient

In [3]:
# -*- coding: utf-8 -*-
"""
聚类系数
"""


import numpy as np
import pandas as pd 
import networkx as nx

    
def data_trans(file):
    '''将代表负边的权重由2变为-1
    输入：原始txt文件
    输出：权值变化后的t.txt文件
    '''
   
    txt = np.loadtxt(file)  
    data = pd.DataFrame(txt)   # 将txt文件变为DataFrame格式
    data.columns = ['n1', 'n2', 'sign']  # 命名列变量
    
    # 将代表负边的权重由2变为-1
    for i in range(len(data)):
        if data['sign'][i] == 2:
            data['sign'][i] = -1
    data.to_csv('t.txt', index = False, sep = '\t', header = None) 
    return 't.txt'


def c_s(G):
    A = nx.to_numpy_matrix(G)  # 原始网络的带符号矩阵
    B = np.dot(A, A)  # 带符号矩阵的平方
    C = np.multiply(A, B)  # 带符号邻接矩阵H乘带符号矩阵的平方
    part_tri = C.sum()

    A1 = abs(A)  # 将带符号的邻接矩阵取绝对值，变为不带符号的邻接矩阵
    B1 = np.dot(A1, A1)  # 不带符号邻接矩阵的平方
    B1_half = np.triu(B1, 1)  # 取B1矩阵的上半三角形
    full_tri = B1_half.sum()*2

    C = part_tri/full_tri  # 全部节点的平均聚类系数
   
    return C

In [4]:
# 生成权值变换后的t.txt文件
data=data_trans('N46edge.txt')

In [6]:
#%%  要等文件生成后才可调用
G = nx.read_edgelist('t.txt',nodetype=float, data=(('weight', float),))
C=c_s(G)
print('average clustering coefficient of the whole network:'+str(C))


average clustering coefficient of the whole network:0.7795847750865051


## embeddedness

In [7]:
# -*- coding: utf-8 -*-
"""
嵌入性
"""


import networkx as nx


def divide_network(G):
    '''将原始网络分别划分为正边网络和负边网络
    输入：G 原始网络
    输出：Gp正边网络
         Gn负边网络
    '''
    
    edge_list = []  # 创建原始网络数据容器
    
    for i, j, weight_data in G.edges(data = True):
        if 'weight' in weight_data:
            # 解除权值字典格式
            edge_list.append([i, j, weight_data['weight']])
             
    positive_edges = []  # 创建正边数据容器
    negitive_edges = []  # 创建负边数据容器
    
    for item in edge_list:
        # 根据权值将正负边数据分别归属到各自容器内
        if item[2] == 1:       
            positive_edges.append(item)
        if item[2] == 2:       
            negitive_edges.append(item)
        
    Gp = nx.Graph()  # 生成正边网络
    Gp.add_weighted_edges_from(positive_edges)
        
    Gn = nx.Graph()  # 生成负边网络
    Gn.add_weighted_edges_from(negitive_edges)
    
    return Gp, Gn


def CommonNeighbor(G, nodeij):
    '''求取节点i,j的共同邻居节点的个数（嵌入性）
    输入：G 对象网络
          nodeij 边数据
    输出：num_cn 共同邻居节点个数
    '''
    
    node_i = nodeij[0]  # 由边获取节点i
    node_j = nodeij[1]  # 由边获取节点j
    # 获取节点i的邻居节点生成器
    neigh_i = set(G.neighbors(node_i))  
    # 获取节点j的邻居节点生成器
    neigh_j = set(G.neighbors(node_j))
    # 获取节点i,j的共同邻居节点生成器
    neigh_ij = neigh_i.intersection(neigh_j)
    # 获取数量
    num_cn = len(neigh_ij)       
    
    return num_cn


def CN_embedding(G, Gpn):
    
    CN_list = []
    # 捕获可能的错误信息
    for item in G.edges():
        try:
            item_CN = CommonNeighbor(Gpn, item)
            # 获取由节点i,j的共同邻居节点数量和两者连边符号列表
            CN_list.append((item_CN, G[item[0]][item[1]]['weight']))
        except:
            pass
    
    # 对共同邻居节点数量进行排序
    CN_list_sort = sorted(CN_list, key=lambda CN_list: CN_list[0])
    degree_list = []
    positive_edge_cn = []
    # 正边两节点没有共同邻居节点的边数
    positve_edge_number = 0
    # 原始网络中 
    sum_edge_number = 0
    item_index = 0
    
    for item in CN_list_sort:
        if item[0] == item_index:
            if item[1] == 1:
                positve_edge_number = positve_edge_number+1
            sum_edge_number = sum_edge_number+1
        elif item[0] > item_index:
            if sum_edge_number == 0:
                degree_list.append(item_index)
                positive_edge_cn.append(0)
            else:  
                degree_list.append(item_index)
                positive_edge_cn.append(float(positve_edge_number)/sum_edge_number)
            item_index = item[0]
            positve_edge_number = 0
            sum_edge_number = 0
    
    return degree_list, positive_edge_cn


# 生成原始网络G
G = nx.read_edgelist('N46edge.txt', nodetype=int, data=(('weight', float),))
# 将原始网络分成正边网络和负边网络
Gp, Gn = divide_network(G)
degree_list_p0, positive_edge_cn_p0 = CN_embedding(G, Gp)
degree_list_n0, positive_edge_cn_n0 = CN_embedding(G, Gn)

print(degree_list_p0,'\n')
print(positive_edge_cn_p0,'\n')
print(degree_list_n0,'\n')
print(positive_edge_cn_n0,'\n')

[0, 1, 2, 3, 4, 5, 10, 17, 18] 

[0.45454545454545453, 0.2857142857142857, 1.0, 0.9, 0.75, 1.0, 1.0, 1.0, 1.0] 

[0, 1, 2] 

[0.6739130434782609, 0.5, 0.25] 



## FMF

In [8]:
# -*- coding: utf-8 -*-
"""
FMF
(节点正负度之差)
"""


import numpy as np
import networkx as nx


def dedivide_network_M(G):
    '''将原始网络分成正边网络和负边网络
      （分别包含原始网络全部节点）
    输入：G原始网络
    输出：Gp正边网络（含所有节点）
         Gn负边网络（含所有节点）
    '''
    
    edge_list = []  # 边数据容器（非字典）
    positive_edges = []  # 正边数据容器
    negative_edges = []  # 负边数据容器
    
    for i, j, weight_data in G.edges(data = True):
        # 消除权值字典结构
        if 'weight' in weight_data:
            edge_list.append([i, j, weight_data['weight']])     
    
    for item in edge_list:
        if item[2] == 1:       
            positive_edges.append(item)
            # M_pos=len(positive_edges)            
        elif item[2] == 2:       
            negative_edges.append(item)
            # M_neg=len(negitive_edges)
    
    # 正边网络                   
    Gp = nx.Graph()
    Gp.add_weighted_edges_from(positive_edges)
    # 负边网络
    Gn = nx.Graph()
    Gn.add_weighted_edges_from(negative_edges)
    
    # 使负边网络包含原始网络所有节点
    for i in Gp:
        if i in Gn.nodes():
            pass
        else:
            Gn.add_node(i)  
    
    # 使正边网络包含原始网络所有节点        
    for j in Gn:
        if j in Gp.nodes():
            pass
        else:
            Gp.add_node(j)             

    return Gp,Gn


def FMF(pos_degree, neg_degree):
    '''计算FMF（节点正负度之差）
    输入：pos_degree 节点正度值容器
         pos_degree 节点负度值容器
    输出：minus 节点FMF值数组
    '''
    
    a = []  # 节点正度值列表
    b = []  # 节点负度值列表
    
    for i in pos_degree:
        a.append(i[1])
    for j in neg_degree:
        b.append(j[1])        

    a_a = np.array(a)  # 节点正度值数组
    b_a = np.array(b)  # 节点负度值数组

    minus = a_a-b_a  # 节点FMF值数组
    
    return minus


def minus_dis(Gp,Gn):
    '''计算原网络节点FMF取值情况列表（排序后）
       与取该FMF值的节点个数占总节点个数的比例
      输入：Gp 正边网络（所有节点）
           Gn 负边网络（所有节点）
      输出：x 节点FMF取值情况列表（排序后）
           y 取该FMF值的节点个数占总节点个数的比例
    '''
    
    pos_degree = Gp.degree()  # 节点正度值字典
    neg_degree = Gn.degree()  # 节点负度值字典
    
    # 解除字典结构并排序
    pos_degree_new = list(sorted(pos_degree, key = lambda pos_degree: pos_degree[0]))
    neg_degree_new = list(sorted(neg_degree, key = lambda neg_degree: neg_degree[0]))

    minus = FMF(pos_degree_new, neg_degree_new)
    x = list(set(list(minus)))
    x.sort()
    y=[]
    
    for i in x:
        y.append(list(minus).count(i)/len(list(minus)))
             
    return x, y


# 生成原网络
G = nx.read_edgelist('N46edge.txt', nodetype = int, data = (('weight', float),))
# 将原网络划分成包含所有节点的正边网络和负边网络
Gp,Gn=dedivide_network_M(G)
# 计算节点FMF值，及取该值节点所占比例
x,y=minus_dis(Gp,Gn)

print('节点FMF值：'+str(x),'\n')
print('取该值节点所占比例：'+str(y),'\n')

节点FMF值：[-6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 18, 19, 20, 27] 

取该值节点所占比例：[0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.03333333333333333, 0.016666666666666666, 0.08333333333333333, 0.03333333333333333, 0.05, 0.016666666666666666, 0.05, 0.03333333333333333, 0.03333333333333333, 0.08333333333333333, 0.016666666666666666, 0.03333333333333333, 0.1, 0.016666666666666666, 0.08333333333333333, 0.016666666666666666, 0.23333333333333334, 0.016666666666666666] 

